In [1]:
pip install efficientnet

     |████████████████████████████████| 50 kB 2.3 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from skimage import io
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import h5py
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
import albumentations
import cv2
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")
import time
from tqdm import tqdm

In [3]:
import tensorflow as tf
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import BatchNormalization, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras import applications
from efficientnet.keras import EfficientNetB0
from keras import callbacks
from keras.models import Sequential
import tensorflow_addons as tfa
from sklearn.metrics import confusion_matrix
import keras.backend as K

Using TensorFlow backend.


In [4]:
#f1 = h5py.File("data_parts.hdf5", "w")
#f1.create_dataset('X_train', data=X_train)
#f1.create_dataset('y_train', data=y_train)
#f1.create_dataset('X_test', data=X_test)
#f1.create_dataset('y_test', data=y_test)
#f1.close
#cohen_kappa_score(y1, y2, weights='quadratic')
#z = np.copy(x)

In [5]:
sz = 256
N = 25
N_row = 5
x_new = sz*N_row
y_new = sz*N_row
input_sh = (x_new,y_new,3)
classes = 6
print(input_sh)

(1280, 1280, 3)


In [6]:
df_train = pd.read_csv('../input/prostate-cancer-grade-assessment/train.csv')
df_train.head()

,image_id,data_provider,isup_grade,gleason_score
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0


In [7]:
dir_p = '../input/panda-dataset-medium-25-256-256/train_medium_25_256_256/'

In [8]:
def read_img_tiles(img_id):
    comb_img = np.zeros(shape=(N,sz,sz,3), dtype='int16')
    for i in range(N):
        path = dir_p +img_id+'_'+str(i)+'.png'
        img_temp = io.imread(path)
        comb_img[i] = img_temp
    comb_img_s = np.copy(comb_img)
    np.random.shuffle(comb_img_s)
    
    comb_img = comb_img.reshape(N_row, N_row, sz,sz,3)
    comb_img = comb_img.transpose(0,2,1,3,4)
    comb_img = comb_img.reshape(x_new,y_new, 3)
    comb_img = (comb_img/255).astype('float32')
    
    comb_img_s = comb_img_s.reshape(N_row, N_row, sz,sz,3)
    comb_img_s = comb_img_s.transpose(0,2,1,3,4)
    comb_img_s = comb_img_s.reshape(x_new,y_new, 3)
    comb_img_s = (comb_img_s/255).astype('float32')
    
    return comb_img, comb_img_s

In [9]:
#img, img_s = read_img_tiles('0005f7aaab2800f6170c399693a96917')

In [10]:
#plt.imshow(img, aspect='auto')
#plt.show()

In [11]:
#plt.imshow(img_s, aspect='auto')
#plt.show()

In [12]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [13]:
def img_to_ndarr(df_train):
    img_id = df_train.image_id.values.tolist()
    y_lst = df_train.isup_grade.values.tolist()
    len_t = len(img_id)
    dataset_y = np.asarray(y_lst)
    dataset_y = convert_to_one_hot(dataset_y, 6).T
    dataset_x1 = np.zeros(shape=(len_t,x_new,y_new,3), dtype='float32')  
    dataset_x2 = np.zeros(shape=(len_t,x_new,y_new,3), dtype='float32')
    for i in range(len_t):
        imd = img_id[i]
        img_arr, img_arr_2 = read_img_tiles(imd)
        dataset_x1[i] = img_arr
        dataset_x2[i] = img_arr_2
    #dataset_x = (dataset_x/255).astype('float32')
    #print(i)
    return dataset_x1, dataset_x2, dataset_y

In [14]:
efficient_net = EfficientNetB0(
    weights='imagenet',
    input_shape=input_sh,
    include_top=False,
)
model = Sequential()
model.add(efficient_net)
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(.10))
model.add(Dense(6, activation='softmax'))

op_1 = tf.keras.optimizers.Adam(learning_rate= 0.0001)

model.compile(optimizer=op_1, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

16809984/16804768 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, 40, 40, 1280)      4049564   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               163968    
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 774       
Total params: 4,214,818
Trainable params: 4,172,546
Non-t

In [15]:
#model2 = tf.keras.models.load_model('../input/pandas-parts/model_en4_low_32_144_9.h5')
#model2.summary()

In [16]:
lr_lst = [3.0000e-04,2.9918e-04,2.9672e-04,2.9266e-04,2.8703e-04,2.7990e-04,2.7135e-04,2.6147e-04,2.5037e-04,2.3817e-04,
         2.2500e-04,2.1101e-04,1.9635e-04,1.8119e-04,1.6568e-04,1.5000e-04,1.3432e-04,1.1881e-04,1.0365e-04,8.8990e-05,
         7.5000e-05,6.1832e-05,4.9630e-05,3.8528e-05,2.8647e-05,2.0096e-05,1.2968e-05,7.3415e-06,3.2779e-06,8.2171e-07]

In [17]:
results = []
loss = []
epoch_lst = [0,1]

In [18]:
for epoch in epoch_lst:
    #print(K.get_value(model.optimizer.learning_rate))
    current_learning_rate = lr_lst[epoch]
    K.set_value(model.optimizer.learning_rate, current_learning_rate)
    print(epoch, K.get_value(model.optimizer.learning_rate))
    for batch in tqdm(range(2654)):
        start = batch*4
        end = start+4
        df = df_train[start:end]
        X_train_1, X_train_2, y_train = img_to_ndarr(df)
        if (batch==0):
            loss = model.train_on_batch(X_train_1, y=y_train, reset_metrics=True)
        else:
            loss = model.train_on_batch(X_train_1, y=y_train, reset_metrics=False)
        loss = model.train_on_batch(X_train_2, y=y_train, reset_metrics=False)


        del X_train_1, X_train_2, y_train
        gc.collect()

    results.append(loss)
    print(results)
    name = 'model_en4_med_256_25_tob_ep_'+str(epoch)+'.h5'
    model.save(name)

  0%|          | 0/2654 [00:00<?, ?it/s]

0 0.0003


100%|██████████| 2654/2654 [2:35:03<00:00,  3.51s/it]  


[[1.4989651441574097, 0.4211418926715851]]


  0%|          | 0/2654 [00:00<?, ?it/s]

1 0.00029918


100%|██████████| 2654/2654 [2:32:33<00:00,  3.45s/it]  


[[1.4989651441574097, 0.4211418926715851], [1.3007872104644775, 0.5012248158454895]]


In [19]:
#results = model.evaluate(X_train, y_train, batch_size=2)

In [20]:
#if (i%6==0):
#name = 'model_en4_med_192_36_decay'+str(i)+'.h5'
#model.save(name)

In [21]:
#df_train.to_csv('split_decay.csv', index=False)